In [2]:
import csv
import datetime
import os
import pandas as pd
import xlsxwriter
from datetime import date

In [3]:
now = datetime.datetime.now()
today = datetime.date.today()
strtoday = now.strftime('%d-%m-%Y')
yesterday = today-datetime.timedelta(days=1)
yday = yesterday.strftime('%d-%m-%Y')
month = now.strftime('%B')
yest = yesterday.strftime('%d')
tod = today.strftime('%d')
numMonth = today.month
d1=date.today()

In [24]:
df = pd.read_excel(month + "_" + yest + "_indent_vs_fulfillment.xlsx",sheet_name='Indent_vs_Fulfillment' )
indent=round(df['Indent Qty(Kgs)'].sum( skipna = True) ,2)
fulfilment=round(df['Fulfill Qty(Kgs)'].sum( skipna = True) ,2)
p_fulfilled=round((fulfilment/indent)*100,2)
p_unfulfilled=round(100-p_fulfilled,2)
df_1=df[df['% Fulfillment']==0]
df_1=df_1.drop('Unnamed: 0',axis=1)
df_1.reset_index(drop=True, inplace=True)
df_2 = df.groupby('SKU').agg(
    {'Rejects':sum}
)
df_2.reset_index(level=0, inplace=True)
df_2=df_2.sort_values(by=['Rejects'],ascending=False).head(5)
df_2.reset_index(drop=True, inplace=True)
#df_2=df_2[['SKU','Rejects']]
df_3 = df.groupby('SKU').agg(
    {'Shortages':sum}
)
df_3.reset_index(level=0, inplace=True)
df_3=df_3.sort_values(by=['Shortages'],ascending=False).head(5)
df_3.reset_index(drop=True, inplace=True)
#df_3=df_3[['SKU','Shortages']]
df_4 = df.groupby('Customer Name').agg(
    {'Rejects':sum,'Shortages':sum}
)
df_4.reset_index(level=0, inplace=True)
df_4=df_4.sort_values(by=['Rejects','Shortages'],ascending=False).head(5)
df_4.reset_index(drop=True, inplace=True)
#df_4=df_4[['Customer Name','Rejects','Shortages']]


In [25]:
import smtplib 
from email.mime.multipart import MIMEMultipart 
from email.mime.text import MIMEText 
from email.mime.base import MIMEBase 
from email import encoders 
from email.mime.image import MIMEImage
   
fromaddr = "shuvo23paul@gmail.com"
#
recipients = ["ravindra.mukund@cloverventures.in","subhankar.paul@cloverventures.in"]
   
# instance of MIMEMultipart 
msg = MIMEMultipart() 
  
# storing the senders email address   
msg['From'] = fromaddr 
  
# storing the receivers email address  
msg['To'] = ", ".join(recipients)
  
# storing the subject  
msg['Subject'] =  "Indent VS Fulfillment dated "+'  '+(d1-datetime.timedelta(days=1)).strftime("%d-%m-%Y")

# string to store the body of the mail 
body="<p>Dear Team</p><p>PFA for <b>"+(d1-datetime.timedelta(days=1)).strftime("%d-%m-%Y")+"</b> Indent VS Fulfillment Report.</p><p> <table border='5' 'bordercolor=#ff0000' cellpadding='5'><tr><td><b>Indent (Kgs)</b></td><td><b>Fulfillment(Kgs)</b></td><td><b>% Fulfilled</b></td><td><b>% Unfulfilled</b></td></tr>"

body+="<tr><td>"+str(indent)+"</td><td>"+str(fulfilment)+"</td><td>"+str(p_fulfilled)+"</td><td>"+str(p_unfulfilled)+"</td></tr>"
    

body+="</table>"

body+='<p><b>100 % Shortage</b></p>'
body+=df_1.to_html()

body+='<p><b>Top 5 SKU Rejects</b></p>'
body+=df_2.to_html()

body+='<p><b>Top 5 SKU Shortages</b></p>'
body+=df_3.to_html()

body+='<p><b>Top 5 Customer Rejects and Shortages</b></p>'
body+=df_4.to_html()

body+="<br><br><p>Thanks and Regards</p><p>Clover Ventures</p>"
# attach the body with the msg instance 
msg.attach(MIMEText(body, "html")) 


# open the file to be sent  
filename = month + "_" + yest + "_indent_vs_fulfillment.xlsx"
attachment = open('/home/subhankar/Desktop/CV/scratch/'+month + "_" + yest + "_indent_vs_fulfillment.xlsx", "rb") 

# instance of MIMEBase and named as p 
p = MIMEBase('application', 'octet-stream') 
  
# To change the payload into encoded form 
p.set_payload((attachment).read()) 
  
# encode into base64 
encoders.encode_base64(p) 
   
p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 

# attach the instance 'p' to instance 'msg' 
msg.attach(p) 
 
# creates SMTP session 
s = smtplib.SMTP('smtp.gmail.com', 587) 
  
# start TLS for security 
s.starttls() 
  
# Authentication 
s.login(fromaddr, "Subhankar#1") 
  
# Converts the Multipart msg into a string 
text = msg.as_string() 
  
# sending the mail 
s.sendmail(fromaddr, recipients, text) 
  
# terminating the session 
s.quit()


(221, b'2.0.0 closing connection 27sm10345401pgk.89 - gsmtp')

In [21]:
df_3

,SKU,Shortages
0,English Cucumber,800
1,Leafy Green Lettuce,788
2,Zucchini - Yellow,770
3,Zucchini - Green,759
4,Iceberg Lettuce,747
